In [1]:
# Disable optional backends to prevent import conflicts (PyTorch-only training)
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
# Reduce CUDA fragmentation for large models
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("✅ Env set: disabled TF/FLAX/TORCHVISION for transformers.")
print("✅ CUDA alloc conf: expandable_segments enabled.")

✅ Env set: disabled TF/FLAX/TORCHVISION for transformers.
✅ CUDA alloc conf: expandable_segments enabled.


# 🚀 GIS代码生成模型训练 - Google Colab (CodeLlama)

本Notebook在Google Colab上训练GIS代码生成模型（**文件级 + CodeLlama**）

**文件级训练** - 模型学习生成完整的工作流而不是单个步骤
- 输入：用户的高层指令（英语/荷兰语，如："Create MS and HS cable objects"）
- 输出：完整的工作流JSON代码（包含所有操作步骤）
- 优势：一次推理生成整个测试脚本
- **模型**：CodeLlama-7B-Instruct（专为代码生成优化）

**使用前准备：**
1. 运行环境：`Runtime > Change runtime type > T4 GPU`（免费）或 `A100 GPU`（Colab Pro）
2. 数据准备：确保已生成训练数据文件
3. 预计时间：4-6小时（T4）/ 1-2小时（A100）

---

## 📋 步骤1：环境设置

In [2]:
# 检查GPU
!nvidia-smi

Thu Jan 22 08:40:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# 安装依赖（约3-5分钟）
print("📦 Installing dependencies...")

# 先锁定关键基础包（避免自动升级）
!pip install -q torch==2.9.0 --no-deps
!pip install -q fsspec==2024.3.1
!pip install -q numpy==2.0.2 --no-deps

# 安装主要训练库（指定兼容版本）
!pip install -q transformers==4.46.0
!pip install -q peft==0.13.0
!pip install -q datasets==2.19.0
!pip install -q "accelerate>=1.0.0"
!pip install -q sentencepiece==0.2.0
!pip install -q tqdm
!pip install -q huggingface-hub==0.26.0

print("✅ Core dependencies installed! If running in Colab, restart runtime after this cell.")

📦 Installing dependencies...
✅ Core dependencies installed! If running in Colab, restart runtime after this cell.


In [ ]:
!pip install -q torchvision

## 💾 步骤2：挂载Google Drive（保存模型）

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# 创建输出目录
!mkdir -p /content/drive/MyDrive/gis-models
print("✅ Google Drive mounted!")

Mounted at /content/drive
✅ Google Drive mounted!


## 📂 步骤3：上传数据

**两种方式任选其一：**

### 方式A：从GitHub克隆仓库（推荐）

In [3]:
# 克隆仓库到当前目录
import os
import subprocess

print("📍 Current directory:", os.getcwd())

# 检查是否已克隆
if os.path.exists('/content/gis-code-ai'):
    print("✅ Repository already exists at /content/gis-code-ai")
    os.chdir('/content/gis-code-ai')
elif os.path.exists('gis-code-ai'):
    print("✅ Repository already exists locally")
    os.chdir('gis-code-ai')
else:
    print("🔄 Cloning repository...")
    result = subprocess.run(
        ['git', 'clone', 'https://github.com/rockyistt/gis-code-ai.git'],
        cwd='/content',
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        os.chdir('/content/gis-code-ai')
        print("✅ Repository cloned successfully")
    else:
        print(f"❌ Clone failed: {result.stderr}")
        raise RuntimeError("Failed to clone repository")

print(f"📍 Working directory: {os.getcwd()}")

# 检查数据文件
print("\n📋 Checking data files...")
required_files = [
    'data/processed/file_level_instructions_weighted_variants_marked.jsonl',
    'data/processed/parsed_workflows.jsonl'
]

all_exist = True
for f in required_files:
    if os.path.exists(f):
        size_mb = os.path.getsize(f) / (1024*1024)
        print(f"✅ {f} ({size_mb:.1f} MB)")
    else:
        print(f"❌ {f}")
        all_exist = False

if all_exist:
    print("\n🎉 All data files ready! Ready to proceed with data preparation")
else:
    print("\n⚠️ Some data files are missing")

📍 Current directory: /content
🔄 Cloning repository...
✅ Repository cloned successfully
📍 Working directory: /content/gis-code-ai

📋 Checking data files...
✅ data/processed/file_level_instructions_weighted_variants_marked.jsonl (2.1 MB)
✅ data/processed/parsed_workflows.jsonl (13.8 MB)

🎉 All data files ready! Ready to proceed with data preparation


## 🔄 步骤4：准备训练数据

## 🔍 诊断：数据过滤分析

在准备训练数据前，先诊断有多少样本在哪个步骤被过滤了

In [ ]:
# 诊断：分析为什么4012条变成1012条
import json
import os
from collections import Counter

print("🔍 数据丢失诊断：为什么4012条变成1012条？")
print("=" * 70)

# 确保在正确目录
if os.path.exists('/content/gis-code-ai'):
    os.chdir('/content/gis-code-ai')
elif os.path.exists('gis-code-ai'):
    os.chdir('gis-code-ai')

instructions_file = 'data/processed/file_level_instructions_weighted_variants_marked.jsonl'
workflows_file = 'data/processed/parsed_workflows.jsonl'

# ============================================================
# 分析指令文件
# ============================================================
print("\n📖 分析指令文件")
print("-" * 70)

instr_lines = []
instr_file_ids = []
instr_valid_json = 0

with open(instructions_file, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f, 1):
        instr_lines.append(line)
        try:
            instr = json.loads(line)
            instr_valid_json += 1
            file_id = instr.get('file_id')
            if file_id:
                instr_file_ids.append(file_id)
        except:
            pass

print(f"   总行数: {len(instr_lines)}")
print(f"   有效JSON: {instr_valid_json}")
print(f"   有file_id的: {len(instr_file_ids)}")

# 检查重复的file_id
instr_id_counts = Counter(instr_file_ids)
instr_duplicates = {fid: count for fid, count in instr_id_counts.items() if count > 1}

print(f"\n   ⚠️ 重复的file_id: {len(instr_duplicates)} 个")
if len(instr_duplicates) > 0:
    print(f"   唯一的file_id: {len(instr_id_counts)} 个")
    print(f"   🔥 这就是问题所在！")
    print(f"\n   重复次数最多的file_id (前10):")
    for fid, count in sorted(instr_duplicates.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"      - '{fid}': 出现 {count} 次")
else:
    print(f"   ✅ 没有重复的file_id")
    print(f"   唯一的file_id: {len(instr_id_counts)} 个")

# ============================================================
# 分析工作流文件
# ============================================================
print("\n" + "=" * 70)
print("📖 分析工作流文件")
print("-" * 70)

workflow_lines = []
workflow_file_ids = []
workflow_valid_json = 0

with open(workflows_file, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f, 1):
        workflow_lines.append(line)
        try:
            workflow = json.loads(line)
            workflow_valid_json += 1
            file_id = workflow.get('file_id')
            if file_id:
                workflow_file_ids.append(file_id)
        except:
            pass

print(f"   总行数: {len(workflow_lines)}")
print(f"   有效JSON: {workflow_valid_json}")
print(f"   有file_id的: {len(workflow_file_ids)}")

# 检查重复的file_id
workflow_id_counts = Counter(workflow_file_ids)
workflow_duplicates = {fid: count for fid, count in workflow_id_counts.items() if count > 1}

print(f"\n   ⚠️ 重复的file_id: {len(workflow_duplicates)} 个")
if len(workflow_duplicates) > 0:
    print(f"   唯一的file_id: {len(workflow_id_counts)} 个")
    print(f"   🔥 这就是问题所在！")
    print(f"\n   重复次数最多的file_id (前10):")
    for fid, count in sorted(workflow_duplicates.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"      - '{fid}': 出现 {count} 次")
else:
    print(f"   ✅ 没有重复的file_id")
    print(f"   唯一的file_id: {len(workflow_id_counts)} 个")

# ============================================================
# 总结
# ============================================================
print("\n" + "=" * 70)
print("📊 诊断结果总结")
print("=" * 70)

print(f"\n指令文件:")
print(f"   - 总行数: {len(instr_lines)}")
print(f"   - 唯一file_id: {len(instr_id_counts)}")
print(f"   - 重复file_id: {len(instr_duplicates)}")
print(f"   - 数据丢失: {len(instr_lines) - len(instr_id_counts)} 行")

print(f"\n工作流文件:")
print(f"   - 总行数: {len(workflow_lines)}")
print(f"   - 唯一file_id: {len(workflow_id_counts)}")
print(f"   - 重复file_id: {len(workflow_duplicates)}")
print(f"   - 数据丢失: {len(workflow_lines) - len(workflow_id_counts)} 行")

# 解释原因
print("\n" + "=" * 70)
print("💡 丢失数据的原因")
print("=" * 70)

if len(instr_duplicates) > 0 or len(workflow_duplicates) > 0:
    print("""
🔥 问题：文件中有重复的file_id！

当前代码使用字典存储：
    instructions[file_id] = instr
    workflows[file_id] = workflow

如果有重复的file_id，后面的会覆盖前面的，导致数据丢失。

举例：
    如果有3条记录都是file_id="test_001"
    字典中只会保留最后一条，前2条被覆盖丢失

解决方案有两种：

方案1: 保留所有数据（推荐）
    改用列表存储所有记录，不去重
    instructions_list = []  # 不用字典
    
方案2: 修复file_id重复问题
    检查数据生成过程，确保每个workflow有唯一的file_id
    可能需要在file_id后面加序号，如 "test_001_v1", "test_001_v2"
""")
else:
    print("""
✅ 没有发现重复的file_id

可能的其他原因：
    1. 某些行没有file_id字段
    2. 某些行JSON格式错误无法解析
    3. 文件读取时编码问题
""")

# 检查匹配情况
common_ids = set(instr_id_counts.keys()) & set(workflow_id_counts.keys())
only_in_instr = set(instr_id_counts.keys()) - set(workflow_id_counts.keys())
only_in_workflow = set(workflow_id_counts.keys()) - set(instr_id_counts.keys())

print("\n" + "=" * 70)
print("🔗 file_id匹配分析")
print("=" * 70)
print(f"   - 两个文件都有: {len(common_ids)} 个")
print(f"   - 仅在指令文件: {len(only_in_instr)} 个")
print(f"   - 仅在工作流文件: {len(only_in_workflow)} 个")

print("\n" + "=" * 70)

In [4]:
import json
from typing import Tuple, List, Dict, Optional

def prepare_file_level_training_data(
    instructions_file: str,
    workflows_file: str,
    output_dir: str = 'data/training',
    max_samples: Optional[int] = None,
    split_ratio: float = 0.9
) -> Tuple[List[Dict], List[Dict]]:
    """
    准备文件级训练数据
    - 输入：file_level_instructions_weighted_variants_marked.jsonl
    - 输出：parsed_workflows.jsonl 中对应的原始JSON
    """
    import os
    import json
    from pathlib import Path

    os.makedirs(output_dir, exist_ok=True)

    # 加载指令
    instructions = {}
    print(f"📖 Loading instructions from {instructions_file}...")
    with open(instructions_file, 'r', encoding='utf-8') as f:
        for line in f:
            instr = json.loads(line)
            file_id = instr.get('file_id')
            if file_id:
                instructions[file_id] = instr
    print(f"✅ Loaded {len(instructions)} instructions")

    # 加载工作流
    workflows = {}
    print(f"📖 Loading workflows from {workflows_file}...")
    with open(workflows_file, 'r', encoding='utf-8') as f:
        for line in f:
            workflow = json.loads(line)
            file_id = workflow.get('file_id')
            if file_id:
                workflows[file_id] = workflow
    print(f"✅ Loaded {len(workflows)} workflows")

    # 配对并创建训练数据
    training_pairs = []
    print("🔄 创建训练对...")

    for file_id, instr in instructions.items():
        if file_id not in workflows:
            continue

        workflow = workflows[file_id]

        # 清理指令
        instruction_text = instr.get('instruction', '').replace('**', '').replace('*', '')
        instruction_text = ' '.join(instruction_text.split()).strip()

        # 构建上下文
        test_app = workflow.get('test_app', '')
        database = workflow.get('database', '')
        total_steps = workflow.get('total_steps', 0)

        context_parts = []
        if test_app:
            context_parts.append(f"Application: {test_app}")
        if database:
            context_parts.append(f"Database: {database}")
        if total_steps > 0:
            context_parts.append(f"Steps: {total_steps}")
        input_context = " | ".join(context_parts)

        # ✅ 直接使用 parsed_workflows.jsonl 中的原始 JSON
        if not workflow.get('steps'):
            continue

        # 直接使用 workflow 对象作为输出
        output_code = json.dumps(workflow, indent=2, ensure_ascii=False)

        # 质量过滤
        if len(instruction_text.split()) < 5 or output_code == '{}':
            continue

        training_pairs.append({
            'instruction': instruction_text,
            'input': input_context,
            'output': output_code
        })

    if max_samples:
        training_pairs = training_pairs[:max_samples]

    print(f"✅ Created {len(training_pairs)} training pairs")

    # 数据集分割
    split_point = int(len(training_pairs) * split_ratio)
    train_data = training_pairs[:split_point]
    val_data = training_pairs[split_point:]

    # 保存为JSON文件
    train_file = os.path.join(output_dir, 'training_data_train.json')
    val_file = os.path.join(output_dir, 'training_data_val.json')

    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=2)
    print(f"💾 Train data saved to {train_file}")

    with open(val_file, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, ensure_ascii=False, indent=2)
    print(f"💾 Val data saved to {val_file}")

    return train_data, val_data

print("✅ prepare_file_level_training_data 函数已定义")

✅ prepare_file_level_training_data 函数已定义


In [5]:
# 运行数据准备（直接调用函数）
import os

# 确保在正确的目录
if os.path.exists('/content/gis-code-ai'):
    os.chdir('/content/gis-code-ai')
elif os.path.exists('gis-code-ai'):
    os.chdir('gis-code-ai')

print("="*70)
print("🔄 开始数据准备...")
print(f"📍 Working directory: {os.getcwd()}")
print("="*70)

train_data, val_data = prepare_file_level_training_data(
    instructions_file='data/processed/file_level_instructions_weighted_variants_marked.jsonl',
    workflows_file='data/processed/parsed_workflows.jsonl',
    output_dir='data/training',
    max_samples=None,  # 使用全部数据，改为整数可限制样本数（如500）
    split_ratio=0.9
)

if train_data is not None:
    print("\n" + "="*70)
    print("🎉 数据准备完成！")
    print(f"📊 数据粒度: 文件级（完整工作流）")
    print(f"📊 训练样本: {len(train_data):,}")
    print(f"📊 验证样本: {len(val_data):,}")
    print("="*70)

    # 显示示例
    if train_data:
        print("\n📝 训练样本示例:")
        sample = train_data[0]
        print(f"Instruction: {sample['instruction'][:100]}...")
        print(f"Input: {sample['input']}")
        print(f"Output preview: {sample['output'][:200]}...")
else:
    print("\n❌ 数据准备失败！请检查上面的错误信息。")

🔄 开始数据准备...
📍 Working directory: /content/gis-code-ai
📖 Loading instructions from data/processed/file_level_instructions_weighted_variants_marked.jsonl...
✅ Loaded 1012 instructions
📖 Loading workflows from data/processed/parsed_workflows.jsonl...
✅ Loaded 1012 workflows
🔄 创建训练对...
✅ Created 1012 training pairs
💾 Train data saved to data/training/training_data_train.json
💾 Val data saved to data/training/training_data_val.json

🎉 数据准备完成！
📊 数据粒度: 文件级（完整工作流）
📊 训练样本: 910
📊 验证样本: 102

📝 训练样本示例:
Instruction: Workflow: create E MS Kabel, E HS Kabel, E LS Kabel in elektra in NRG Beheerkaart Elektra MS...
Input: Application: NRG Beheerkaart Elektra MS | Steps: 7
Output preview: {
  "file_id": "template_insert_kabels_ms_ls_hs_pretty",
  "file_path": "data\\raw\\template\\template_insert_kabels_ms_ls_hs_pretty.json",
  "is_high_quality": true,
  "test_env": "TST",
  "test_app"...


## 🚀 步骤5：开始训练

In [9]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset

print("🔧 Setting up training with CodeLlama...")

# ============================================================
# 配置参数 - CodeLlama (英语/荷兰语代码生成优化)
# ============================================================

MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"
OUTPUT_DIR = "/content/drive/MyDrive/gis-models/codellama-gis-lora"  # 保存到Google Drive
TRAIN_FILE = "data/training/training_data_train.json"
VAL_FILE = "data/training/training_data_val.json"

# 训练参数（T4 GPU内存优化版 - 14GB显存限制）
NUM_EPOCHS = 3
BATCH_SIZE = 1  # Per-device batch size
GRADIENT_ACCUMULATION = 2  # Reduced from 4 → effective batch = 2
LEARNING_RATE = 2e-4  # CodeLlama推荐学习率
MAX_LENGTH = 512  # Reduced from 768 → significant memory savings

# LoRA参数（代码生成任务优化 - T4优化版）
LORA_R = 32  # Reduced from 64 → less trainable parameters
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

print(f"📦 Model: {MODEL_NAME}")
print(f"💾 Output: {OUTPUT_DIR}")
print(f"📊 Data: FILE-LEVEL (complete workflows)")
print(f"📈 Epochs: {NUM_EPOCHS}, Batch: {BATCH_SIZE}, Accumulation: {GRADIENT_ACCUMULATION}, LR: {LEARNING_RATE}")
print(f"📄 Max Length: {MAX_LENGTH}")
print(f"🎯 LoRA Rank: {LORA_R} (T4-optimized)")
print(f"🎯 Optimized for: T4 GPU (14GB VRAM) - memory-efficient config")

🔧 Setting up training with CodeLlama...
📦 Model: codellama/CodeLlama-7b-Instruct-hf
💾 Output: /content/drive/MyDrive/gis-models/codellama-gis-lora
📊 Data: FILE-LEVEL (complete workflows)
📈 Epochs: 3, Batch: 1, Accumulation: 2, LR: 0.0002
📄 Max Length: 512
🎯 LoRA Rank: 32 (T4-optimized)
🎯 Optimized for: T4 GPU (14GB VRAM) - memory-efficient config


In [10]:
# 加载tokenizer (CodeLlama)
print("📖 Loading CodeLlama tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side="right"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Tokenizer loaded: vocab_size={len(tokenizer)}")

📖 Loading CodeLlama tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

✅ Tokenizer loaded: vocab_size=32016


In [11]:
# ⚠️ 必须在model loading前运行！升级关键库
print("🔧 Upgrading transformers, accelerate, and installing bitsandbytes...")

# 升级transformers到兼容版本
!pip install -q --upgrade transformers==4.46.0

# 升级accelerate到最新稳定版修复optimizer.train()错误
!pip install -q --upgrade accelerate>=1.0.0

# 安装bitsandbytes用于8-bit优化器（节省显存）
!pip install -q bitsandbytes

print("✅ Libraries upgraded!")
print(f"  transformers: 4.46.0")
print(f"  accelerate: >=1.0.0 (latest stable, fixes optimizer.train() bug)")
print(f"  bitsandbytes: installed (8-bit optimizer for memory efficiency)")

🔧 Upgrading transformers, accelerate, and installing bitsandbytes...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.8 MB/s eta 0:00:00
✅ Libraries upgraded!
  transformers: 4.46.0
  accelerate: >=1.0.0 (latest stable, fixes optimizer.train() bug)
  bitsandbytes: installed (8-bit optimizer for memory efficiency)


In [12]:
# 确保导入了必要的库
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

# 使用float16加载模型（不量化，稳定适配T4显存）
print("🤖 Loading CodeLlama-7B with float16 and device offloading...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
)

# 训练中必须禁用缓存以配合梯度检查点
model.config.use_cache = False

# 启用梯度检查点（节省显存）
model.gradient_checkpointing_enable()

print("✅ CodeLlama base model loaded (float16 with device offloading)")

# 应用LoRA
print("🔧 Applying LoRA...")
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ LoRA applied!")

🤖 Loading CodeLlama-7B with float16 and device offloading...


config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ CodeLlama base model loaded (float16 with device offloading)
🔧 Applying LoRA...
trainable params: 79,953,920 || all params: 6,818,500,608 || trainable%: 1.1726
✅ LoRA applied!


In [13]:
from datasets import load_dataset

# 准备数据集
print("📊 Preparing datasets...")

train_data = load_dataset('json', data_files=TRAIN_FILE, split='train')
eval_data = load_dataset('json', data_files=VAL_FILE, split='train')

print(f"  Train: {len(train_data)} samples")
print(f"  Val: {len(eval_data)} samples")

# 格式化prompt (CodeLlama优化格式)
def format_prompt(example):
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']

    # CodeLlama更适合直接的代码生成格式
    if input_text:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {input_text}

JSON Code:
{output}"""
    else:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}

JSON Code:
{output}"""

    return {"text": prompt}

train_data = train_data.map(format_prompt, remove_columns=train_data.column_names)
eval_data = eval_data.map(format_prompt, remove_columns=eval_data.column_names)

# Tokenize
def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        return_tensors=None
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔄 Tokenizing...")
train_dataset = train_data.map(
    tokenize_function,
    batched=True,
    remove_columns=train_data.column_names,
    desc="Tokenizing train"
)

eval_dataset = eval_data.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_data.column_names,
    desc="Tokenizing val"
)

print("✅ Datasets ready!")

📊 Preparing datasets...


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

  Train: 910 samples
  Val: 102 samples


Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

🔄 Tokenizing...


Tokenizing train:   0%|          | 0/910 [00:00<?, ? examples/s]

Tokenizing val:   0%|          | 0/102 [00:00<?, ? examples/s]

✅ Datasets ready!


In [14]:
# 配置训练
print("⚙️ Configuring training (T4-optimized)...")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    warmup_steps=50,  # Reduced from 100
    logging_steps=10,
    save_steps=300,  # Reduced from 500
    eval_steps=300,  # Reduced from 500
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    fp16=True,  # Enable fp16 compute
    bf16=False,
    optim="adamw_8bit",  # 8-bit AdamW for memory efficiency (requires bitsandbytes)
    lr_scheduler_type="cosine",
    save_total_limit=2,  # Reduced from 3 to save disk space
    report_to="none",
    logging_dir=f"{OUTPUT_DIR}/logs",
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,
    push_to_hub=False,
    gradient_checkpointing=True,  # Memory savings via checkpointing
    max_grad_norm=1.0,  # Gradient clipping
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,  # Use processing_class instead of tokenizer (new API)
    data_collator=data_collator,
)

print("✅ Trainer ready!")
print("\n" + "="*70)
print("🚀 Starting training (T4-optimized, may take 1-2 hours)...")
print("="*70)

# 训练前清理显存，减少碎片
import gc
gc.collect()
torch.cuda.empty_cache()

# 开始训练
trainer.train()

print("\n" + "="*70)
print("🎉 Training completed!")
print("="*70)

⚙️ Configuring training (T4-optimized)...
✅ Trainer ready!

🚀 Starting training (T4-optimized, may take 1-2 hours)...


Step,Training Loss,Validation Loss
300,0.050500,0.047307
600,0.043500,0.044583
900,0.047400,0.044165
1200,0.037400,0.043688



🎉 Training completed!


In [15]:
# 🧪 验证集快速抽样测试（查看生成效果）
import torch, json, textwrap
from random import sample

print("\n" + "="*70)
print("🧪 Quick sanity check on validation set (5 samples)")
print("="*70)

model.eval()
SAMPLE_N = min(5, len(eval_data))
indices = sample(range(len(eval_data)), SAMPLE_N)

for idx in indices:
    item = eval_data[int(idx)]
    instruction = item.get("instruction", "")
    context = item.get("input", "")
    prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {context}

JSON Code:
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 仅截取JSON部分
    if "JSON Code:" in generated:
        generated = generated.split("JSON Code:")[-1].strip()
    print("\n" + "-"*70)
    print(f"Sample #{idx}")
    print("Instruction:", textwrap.shorten(instruction, width=140))
    print("Context:", textwrap.shorten(context, width=140))
    print("Generated (truncated):")
    print(textwrap.shorten(generated.replace("\n", " "), width=400))


🧪 Quick sanity check on validation set (5 samples)

----------------------------------------------------------------------
Sample #81
Instruction: 
Context: 
Generated (truncated):
{ "workflow": { "metadata": { "test_app": "elektra", "database": "", "total_steps": 4 }, "steps": [ { "step_index": 0, "database": ":elektra", "object": "E Station Terrein", "object_id": "Passed", "module": "Editor(s)", "method": "Verify Field", "command": "Execute Object Control Testcommand", "test_data": { "create": {}, "update": {}, "editor": { "Object": "E Station Terrein", "Type": [...]

----------------------------------------------------------------------
Sample #14
Instruction: 
Context: 
Generated (truncated):
{ "workflow": { "metadata": { "test_app": "elektra", "database": "", "total_steps": 3 }, "steps": [ { "step_index": 0, "database": ":elektra", "object": "E SO Hulpspanning Installatie FP", "object_id": "Passed", "module": "Editor(s)", "method": "Switch Spatial Context", "command": "Execute Ob

In [16]:
# 📊 结构化成功率指标（验证集）
import numpy as np

print("\n" + "="*70)
print("📊 Validation set structural success metrics")
print("="*70)

REQUIRED_TOP_KEYS = ["workflow"]
REQUIRED_WORKFLOW_KEYS = ["steps"]
REQUIRED_STEP_KEYS = ["type", "params"]

def extract_json(text):
    if "{" not in text:
        return text
    start = text.find("{")
    end = text.rfind("}")
    return text[start:end+1] if end >= start else text

def validate_one(text):
    result = {"json_ok": False, "ok": False, "missing_top": [], "missing_workflow": [], "missing_steps": [], "steps": 0}
    try:
        obj = json.loads(text)
        result["json_ok"] = True
    except Exception:
        return result
    missing_top = [k for k in REQUIRED_TOP_KEYS if k not in obj]
    wf = obj.get("workflow", {})
    missing_wf = [k for k in REQUIRED_WORKFLOW_KEYS if k not in wf]
    steps = wf.get("steps", []) if isinstance(wf, dict) else []
    step_missing = []
    for s in steps:
        if isinstance(s, dict):
            miss = [k for k in REQUIRED_STEP_KEYS if k not in s]
        else:
            miss = REQUIRED_STEP_KEYS
        step_missing.append(miss)
    result.update({
        "missing_top": missing_top,
        "missing_workflow": missing_wf,
        "missing_steps": step_missing,
        "steps": len(steps),
    })
    ok = (len(missing_top)==0 and len(missing_wf)==0 and all(len(m)==0 for m in step_missing) and len(steps)>0)
    result["ok"] = ok
    return result

MAX_EVAL = min(100, len(eval_data))
success = 0
json_ok = 0
step_counts = []
fail_reasons = []

for i in range(MAX_EVAL):
    item = eval_data[int(i)]
    instruction = item.get("instruction", "")
    context = item.get("input", "")
    prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {context}

JSON Code:
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "JSON Code:" in text:
        text = text.split("JSON Code:")[-1].strip()
    text = extract_json(text)
    res = validate_one(text)
    json_ok += int(res["json_ok"])
    success += int(res["ok"])
    step_counts.append(res.get("steps", 0))
    if not res["ok"]:
        reason = []
        if not res["json_ok"]: reason.append("json_parse")
        if res["missing_top"]: reason.append(f"miss_top:{','.join(res['missing_top'])}")
        if res["missing_workflow"]: reason.append(f"miss_workflow:{','.join(res['missing_workflow'])}")
        if any(res["missing_steps"]): reason.append("miss_step_fields")
        if res.get("steps",0)==0: reason.append("no_steps")
        fail_reasons.append(",".join(reason) or "unknown")

N = MAX_EVAL
json_rate = json_ok / N if N else 0
success_rate = success / N if N else 0
avg_steps = float(np.mean(step_counts)) if step_counts else 0
print(f"Samples evaluated: {N}")
print(f"JSON parse success: {json_rate:.1%}")
print(f"Full structural success: {success_rate:.1%}")
print(f"Average steps (all): {avg_steps:.2f}")

from collections import Counter
if fail_reasons:
    top_fail = Counter(fail_reasons).most_common(5)
    print("Top failure reasons:")
    for r,c in top_fail:
        print(f"  {r}: {c}")
else:
    print("No failures recorded")


📊 Validation set structural success metrics
Samples evaluated: 100
JSON parse success: 1.0%
Full structural success: 0.0%
Average steps (all): 0.01
Top failure reasons:
  json_parse,no_steps: 99
  miss_step_fields: 1


In [ ]:
# 保存模型
print("💾 Saving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Model saved to {OUTPUT_DIR}")

# 保存训练信息
import json
training_info = {
    "model_name": MODEL_NAME,
    "num_epochs": NUM_EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "train_samples": len(train_dataset),
    "val_samples": len(eval_dataset),
}

with open(f"{OUTPUT_DIR}/training_info.json", 'w') as f:
    json.dump(training_info, f, indent=2)

print("\n📊 Training Summary:")
for key, value in training_info.items():
    print(f"  {key}: {value}")

💾 Saving model...
✅ Model saved to /content/drive/MyDrive/gis-models/codellama-gis-lora

📊 Training Summary:
  model_name: codellama/CodeLlama-7b-Instruct-hf
  num_epochs: 3
  batch_size: 1
  learning_rate: 0.0002
  lora_r: 32
  lora_alpha: 16
  train_samples: 910
  val_samples: 102


## 🧪 步骤6：测试模型

In [ ]:
# 快速测试 (CodeLlama)
print("🧪 Testing CodeLlama model inference...")

test_instruction = "Create a new MS cable object at coordinates (186355533, 439556907)"
test_context = "Application: PowerGrid | Database: ND | Steps: 5"

prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {test_instruction}
Context: {test_context}

JSON Code:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print("\n🔮 Generating...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.split("JSON Code:")[-1].strip()

print("\n" + "="*70)
print("📝 Test Result:")
print("="*70)
print(f"Instruction: {test_instruction}")
print(f"Context: {test_context}")
print(f"\nGenerated Output:\n{response[:500]}...")
print("="*70)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


🧪 Testing CodeLlama model inference...

🔮 Generating...

📝 Test Result:
Instruction: Create a new MS cable object at coordinates (186355533, 439556907)
Context: Application: PowerGrid | Database: ND | Steps: 5

Generated Output:
{
  "workflow": {
    "metadata": {
      "test_app": "PowerGrid",
      "database": "ND",
      "total_steps": 5
    },
    "steps": [
      {
        "step_index": 0,
        "database": ":catalogus",
        "object": "E MS Kabel",
        "object_id": "Passed",
        "module": "Editor(s)",
        "method": "Open Object",
        "command": "Execute Object Control Testcommand",
        "test_data": {
          "create": {},
          "update": {},
          "editor": {
            "Object"...


## 📦 步骤7：下载模型（可选）

In [ ]:
# 打包模型用于下载
!cd /content/drive/MyDrive/gis-models && zip -r codellama-gis-lora.zip codellama-gis-lora/
print("✅ Model zipped!")
print(f"📦 Location: /content/drive/MyDrive/gis-models/codellama-gis-lora.zip")
print("💡 You can download it from Google Drive")

  adding: codellama-gis-lora/ (stored 0%)
  adding: codellama-gis-lora/checkpoint-1365/ (stored 0%)
  adding: codellama-gis-lora/checkpoint-1365/README.md (deflated 66%)
  adding: codellama-gis-lora/checkpoint-1365/adapter_model.safetensors (deflated 7%)
  adding: codellama-gis-lora/checkpoint-1365/adapter_config.json (deflated 54%)
  adding: codellama-gis-lora/checkpoint-1365/tokenizer_config.json (deflated 73%)
  adding: codellama-gis-lora/checkpoint-1365/special_tokens_map.json (deflated 68%)
  adding: codellama-gis-lora/checkpoint-1365/tokenizer.model (deflated 55%)
  adding: codellama-gis-lora/checkpoint-1365/tokenizer.json (deflated 85%)
  adding: codellama-gis-lora/checkpoint-1365/training_args.bin (deflated 54%)
  adding: codellama-gis-lora/checkpoint-1365/optimizer.pt (deflated 10%)
  adding: codellama-gis-lora/checkpoint-1365/scheduler.pt (deflated 62%)
  adding: codellama-gis-lora/checkpoint-1365/rng_state.pth (deflated 26%)
  adding: codellama-gis-lora/checkpoint-1365/train

## 🎯 下一步

训练完成后，你可以：

1. **在Colab中继续测试**：使用上面的测试单元格
2. **下载模型**：从Google Drive下载打包的模型
3. **本地部署**：将模型下载到本地进行推理
4. **评估模型**：运行完整的评估脚本

### 本地使用模型

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 加载CodeLlama基座模型
base_model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
model = PeftModel.from_pretrained(base_model, "path/to/codellama-gis-lora")
tokenizer = AutoTokenizer.from_pretrained("path/to/codellama-gis-lora")

# 推理
# ...
```

---

**训练参数调优建议：**
- T4 GPU: batch_size=4, gradient_accumulation=4
- A100 GPU: batch_size=8-12, gradient_accumulation=2-4
- 如果显存不足：减小batch_size或max_length
- 训练时间估计：T4约4-6小时，A100约1-2小时

**🎉 恭喜！你已经完成了GIS代码生成模型的训练（CodeLlama版）！**

## 🧪 步骤8: 加载模型和评估

**已完成模型训练？现在让我们加载它并在测试集上评估性能！**

两种方式选择:
1. **快速推理测试** (下面单元格)
2. **完整评估** (后续单元格)


# ⚡ 快速开始：加载已训练模型（跳过训练步骤）

**如果你已经完成了模型训练并保存到Google Drive，可以直接从这里开始！**

- 无需运行步骤1-7的训练代码
- 直接加载模型并在测试集上评估效果
- 约需5-10分钟完成

**操作流程：**
1. 运行"挂载Google Drive"
2. 运行"安装依赖"（如果提示缺少包）
3. 运行"内存预检查和清理"
4. 运行"加载已训练的模型"（使用8-bit量化减少内存！）
5. 运行"快速推理测试"  
6. 运行"在测试集上完整评估"

---

## ⚠️ 内存优化说明

这个版本已经过**内存优化**，适合在Colab免费版（T4 GPU，12GB VRAM）上运行：

| 优化项 | 效果 | 说明 |
|--------|------|------|
| **8-bit量化** | 减少40%内存 | 将float32/float16权重量化为int8，精度损失<1% |
| **device_map="auto"** | 自动分布 | 模型层自动分布在GPU和CPU间，防止单GPU溢出 |
| **max_new_tokens=256** | 减少显存占用 | 限制生成长度，减少KV缓存 |
| **内存预检查** | 提前发现问题 | 加载前检查可用内存，避免OOM中途crash |
| **逐步清理** | 释放显存 | 推理后立即释放tensor，调用torch.cuda.empty_cache() |

如果仍然显存不足，可以尝试：
1. **减少评估样本数** → 修改 `NUM_EVAL_SAMPLES = 20` 
2. **进一步减少max_tokens** → 改为 `max_tokens=128`
3. **使用4-bit量化** → 修改 `load_in_4bit=True` （需要bitsandbytes）
4. **CPU推理** → 使用CPU进行推理（速度很慢但内存充足）

---

In [ ]:
# 挂载Google Drive（获取已保存的模型）
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive已挂载！")


In [ ]:
# ⚠️ 内存预检查和清理（必须在加载模型前运行！）
import psutil
import gc
import torch

print("💾 系统内存检查...")
print("=" * 70)

# 获取系统内存信息
mem = psutil.virtual_memory()
print(f"系统总内存: {mem.total / (1024**3):.1f} GB")
print(f"已使用: {mem.used / (1024**3):.1f} GB ({mem.percent:.1f}%)")
print(f"可用: {mem.available / (1024**3):.1f} GB")

# 获取GPU内存信息（如果有GPU）
if torch.cuda.is_available():
    print(f"\n🖥️  GPU信息:")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU总内存: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
    print(f"GPU已用: {torch.cuda.memory_allocated() / (1024**3):.1f} GB")
    print(f"GPU保留: {torch.cuda.memory_reserved() / (1024**3):.1f} GB")
else:
    print(f"\n❌ 未检测到GPU，模型推理会很慢!")

# 如果内存使用超过80%，进行清理
if mem.percent > 80:
    print(f"\n⚠️ 系统内存使用过高 ({mem.percent:.1f}%)，进行清理...")
    gc.collect()
    torch.cuda.empty_cache()
    mem = psutil.virtual_memory()
    print(f"清理后可用: {mem.available / (1024**3):.1f} GB")

# 如果可用内存不足12GB，发出警告
if mem.available < 12 * (1024**3):
    print(f"\n⚠️ 警告：可用内存少于12GB，模型加载可能会失败！")
    print(f"建议: ")
    print(f"  1. 关闭其他标签页或程序")
    print(f"  2. 运行以下代码清理更多内存:")
    print(f"     gc.collect()")
    print(f"     torch.cuda.empty_cache()")
else:
    print(f"\n✅ 内存充足，可以继续加载模型")

print("=" * 70)


In [ ]:
# 安装内存监控工具
print("📦 安装依赖...")
!pip install -q psutil
print("✅ 已安装psutil\n")


## 📦 第1步：加载已训练的模型

In [ ]:
import torch
import json
import os
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import numpy as np
from tqdm import tqdm
import gc

print("🔍 检查模型文件...")

MODEL_PATH = "/content/drive/MyDrive/gis-models/codellama-gis-lora"
BASE_MODEL = "codellama/CodeLlama-7b-Instruct-hf"

# 验证模型路径
if not Path(MODEL_PATH).exists():
    print(f"❌ 模型路径不存在: {MODEL_PATH}")
    print(f"\n可能的原因:")
    print(f"  1. 模型还没有训练完成")
    print(f"  2. 模型保存到了不同的位置")
    print(f"\n请检查以下位置:")
    gis_models = "/content/drive/MyDrive/gis-models"
    if Path(gis_models).exists():
        print(f"  已有的目录: {os.listdir(gis_models)}")
    raise FileNotFoundError(f"模型文件未找到: {MODEL_PATH}")

print(f"✅ 模型路径确认: {MODEL_PATH}\n")

# 清理内存
print("🧹 清理缓存...")
gc.collect()
torch.cuda.empty_cache()
print("✅ 内存已清理\n")

# 加载Tokenizer
print("📖 加载Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    padding_side="right",
    local_files_only=True,
    trust_remote_code=True,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer加载完成\n")

# 加载基础模型（内存优化版本）
print("🤖 加载基础模型(CodeLlama-7B - 内存优化模式)...")
device = "cuda" if torch.cuda.is_available() else "cpu"

# ⚠️ 关键优化：使用8-bit量化减少40%内存占用
try:
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",  # 自动设备映射，智能分布模型权重
        load_in_8bit=True,  # 🔥 8-bit量化：14GB → 8GB内存
        low_cpu_mem_usage=True,
    )
    print(f"✅ 基础模型加载完成 (8-bit量化，设备: {device})")
    using_8bit = True
except Exception as e:
    print(f"⚠️ 8-bit量化失败: {e}")
    print(f"🔄 尝试使用标准加载（可能仍然会OOM）...")
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",  # 仍然使用auto映射
        low_cpu_mem_usage=True,
    )
    using_8bit = False
    print(f"✅ 基础模型加载完成 (设备: {device})")

base_model.config.use_cache = False
print()

# 加载LoRA权重
print("🔧 加载LoRA权重...")
try:
    model = PeftModel.from_pretrained(
        base_model,
        MODEL_PATH,
        is_trainable=False,
        local_files_only=True,
    )
    model.eval()
    print("✅ LoRA权重加载完成\n")
except Exception as e:
    print(f"❌ LoRA加载失败: {e}")
    print(f"尝试使用基础模型进行推理（不使用LoRA）...")
    model = base_model
    print()

print("=" * 70)
print("🎉 模型加载成功！")
print("=" * 70)
print(f"📊 配置信息:")
print(f"  - 基础模型: CodeLlama-7B-Instruct")
print(f"  - 量化方式: {'8-bit (低内存)' if using_8bit else 'float16'}")
print(f"  - 设备映射: 自动分布式加载")
print(f"  - LoRA: 已应用")
print("=" * 70)


## 🧪 第2步：快速推理测试

In [ ]:
# 定义优化的推理函数（内存友好）
def generate_code(instruction, context="", max_tokens=256):
    """
    使用加载的模型生成代码（内存优化版本）
    - 减少max_new_tokens来节省显存
    - 自动释放不需要的中间变量
    """
    if context:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {context}

JSON Code:
"""
    else:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}

JSON Code:
"""

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(model.device)

    try:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,  # 减少token数来节省内存
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
                repetition_penalty=1.1,  # 防止重复，减少生成长度
            )

        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        code = text.split("JSON Code:")[-1].strip()
        
        # 释放输入tensor
        del inputs, outputs
        torch.cuda.empty_cache()
        
        return code
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            print(f"❌ 显存不足！")
            print(f"解决方案:")
            print(f"  1. 减少 max_tokens (当前: {max_tokens})")
            print(f"  2. 减少 batch_size")
            print(f"  3. 重启kernel并只加载必要的模型部分")
            torch.cuda.empty_cache()
            return None
        else:
            raise

# 测试案例
print("🧪 运行推理测试 (2个示例)...\n")

test_cases = [
    {
        "instruction": "Create a new MS cable object at coordinates (186355533, 439556907)",
        "context": "Application: PowerGrid | Database: ND | Steps: 5"
    },
    {
        "instruction": "Open object in editor and verify fields",
        "context": "Application: NRG Elektra | Database: elektra | Steps: 3"
    }
]

for i, test in enumerate(test_cases, 1):
    print(f"{'='*70}")
    print(f"📝 测试案例 {i}:")
    print(f"{'='*70}")
    print(f"指令: {test['instruction']}")
    print(f"上下文: {test['context']}")
    print()

    result = generate_code(test['instruction'], test['context'], max_tokens=256)
    
    if result is None:
        print("⚠️ 生成失败，跳过")
        continue

    # 检查JSON有效性
    try:
        json_obj = json.loads(result)
        print(f"✅ JSON有效\n")
        print(f"💻 生成的JSON代码 (前300字符):")
        print(f"{result[:300]}...")
    except Exception as e:
        print(f"⚠️ JSON格式问题: {e}\n")
        print(f"💻 生成内容 (前300字符):")
        print(f"{result[:300]}...")
    
    print()

print("=" * 70)


## 📊 第3步：在测试集上完整评估模型

In [ ]:
# 检查并加载测试数据
print("📂 加载测试数据...\n")

# 确保在正确的目录
if os.path.exists('/content/gis-code-ai'):
    os.chdir('/content/gis-code-ai')
elif os.path.exists('gis-code-ai'):
    os.chdir('gis-code-ai')

TEST_DATA_FILE = 'data/training/training_data_val.json'

if not os.path.exists(TEST_DATA_FILE):
    print(f"❌ 测试数据文件不存在: {TEST_DATA_FILE}")
    print(f"\n请确保已运行过数据准备步骤（步骤4）")
    raise FileNotFoundError(f"测试数据文件未找到: {TEST_DATA_FILE}")

with open(TEST_DATA_FILE, 'r', encoding='utf-8') as f:
    test_data = json.load(f)

print(f"✅ 加载了 {len(test_data)} 个测试样本\n")

# 定义评估指标
def is_valid_json(text):
    """检查是否是有效的JSON"""
    try:
        json.loads(text)
        return True
    except:
        return False

def get_step_count(text):
    """从生成的代码中提取步骤数"""
    try:
        obj = json.loads(text)
        return len(obj.get("workflow", {}).get("steps", []))
    except:
        return 0

# 开始评估
NUM_EVAL_SAMPLES = min(50, len(test_data))  # 评估前50个样本（内存友好）
print(f"🧪 开始在测试集上评估 ({NUM_EVAL_SAMPLES} 个样本)...\n")
print(f"💡 提示：评估样本数已减少到50个以节省内存。如需完整评估，可修改NUM_EVAL_SAMPLES\n")

json_valid_count = 0
step_counts = []
all_results = []
errors = []
failed_samples = 0

for i in tqdm(range(NUM_EVAL_SAMPLES), desc="评估进度", ncols=80):
    sample = test_data[i]

    try:
        instruction = sample.get("instruction", "")
        context = sample.get("input", "")

        # 生成代码（使用较小的max_tokens来节省内存）
        generated_code = generate_code(instruction, context, max_tokens=200)
        
        if generated_code is None:
            # 显存不足，停止评估
            print(f"\n⚠️ 显存不足，停止评估。已完成 {i}/{NUM_EVAL_SAMPLES} 个样本")
            break

        # 计算指标
        is_valid = is_valid_json(generated_code)
        step_count = get_step_count(generated_code)

        if is_valid:
            json_valid_count += 1

        step_counts.append(step_count)

        all_results.append({
            "instruction": instruction,
            "json_valid": is_valid,
            "step_count": step_count,
            "generated_code": generated_code[:200]  # 保留前200字符用于调试
        })
    except Exception as e:
        step_counts.append(0)
        errors.append(str(e))
        failed_samples += 1
        all_results.append({
            "instruction": instruction,
            "json_valid": False,
            "step_count": 0,
            "error": str(e)
        })

# 计算统计
print("\n" + "=" * 70)
print("📊 模型评估结果摘要")
print("=" * 70)

total_samples = len(all_results)
json_valid_rate = json_valid_count / total_samples if total_samples > 0 else 0

print(f"\n✅ JSON有效性:")
print(f"   有效: {json_valid_count}/{total_samples} ({json_valid_rate:.1%})")

if step_counts:
    print(f"\n📍 步骤数统计:")
    print(f"   平均: {np.mean(step_counts):.1f}")
    print(f"   中位: {np.median(step_counts):.1f}")
    print(f"   最大: {int(np.max(step_counts)) if len(step_counts) > 0 else 0}")
    print(f"   最小: {int(np.min(step_counts)) if len(step_counts) > 0 else 0}")

if failed_samples > 0:
    print(f"\n⚠️ 失败统计:")
    print(f"   失败样本数: {failed_samples}")
    print(f"   失败率: {failed_samples/total_samples:.1%}")

# 评分
quality_score = "⭐⭐⭐⭐" if json_valid_rate > 0.7 else "⭐⭐⭐" if json_valid_rate > 0.5 else "⭐⭐"
print(f"\n🎯 总体质量评估: {quality_score}")

print("\n" + "=" * 70)
print("✅ 评估完成！")
print("=" * 70)


## 💾 导出评估结果

保存评估结果到文件，用于后续分析

In [ ]:
# 保存评估结果
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = f"model_evaluation_results_{timestamp}.json"

evaluation_summary = {
    "timestamp": timestamp,
    "total_samples_evaluated": total_samples,
    "json_valid_count": json_valid_count,
    "json_valid_rate": float(json_valid_rate),
    "step_count_stats": {
        "mean": float(np.mean(step_counts)),
        "median": float(np.median(step_counts)),
        "max": int(np.max(step_counts)),
        "min": int(np.min(step_counts))
    },
    "quality_score": quality_score,
    "sample_results": all_results
}

with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(evaluation_summary, f, ensure_ascii=False, indent=2)

print(f"💾 评估结果已保存到: {results_file}")
print(f"\n📊 结果摘要:")
print(f"   JSON有效率: {json_valid_rate:.1%}")
print(f"   平均步骤数: {np.mean(step_counts):.1f}")
print(f"   质量评分: {quality_score}")


## 🆘 故障排查 (Troubleshooting)

### 问题1: RuntimeError: CUDA out of memory

**原因：** GPU显存不足（典型12GB T4或40GB A100）

**解决方案：**
1. **立即修复** → 重启kernel：`Ctrl+M` → 选择"Restart runtime"
2. **减少样本数** → 修改 `NUM_EVAL_SAMPLES = 20`
3. **减少生成长度** → 修改 `max_tokens=128`
4. **启用4-bit量化** → 修改 `load_in_4bit=True`
5. **使用CPU推理** → 改 `device = "cpu"` （很慢但有效）

```python
# 应急清理
import gc
gc.collect()
torch.cuda.empty_cache()

# 然后重新加载模型，用更激进的设置
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_4bit=True,  # 4-bit量化，更激进
    device_map="auto",
)
```

### 问题2: Model file not found

**原因：** 模型没有保存或路径错误

**排查步骤：**
```python
import os
# 检查模型是否存在
model_path = "/content/drive/MyDrive/gis-models/codellama-gis-lora"
if os.path.exists(model_path):
    print("✅ 模型存在")
    print(os.listdir(model_path))
else:
    print("❌ 模型不存在，需要重新训练")
```

### 问题3: Kernel crashes during loading

**原因：** 内存不足导致kernel crash（不是RuntimeError，而是直接崩溃）

**解决方案：**
1. 重启kernel
2. **不要在同一kernel中连续运行多个任务** → 完成评估后关闭模型：
```python
del model, base_model, tokenizer
torch.cuda.empty_cache()
```
3. 使用T4 GPU而不是其他配置

### 问题4: JSON解析失败

**原因：** 模型生成的不是有效JSON格式

**检查方法：**
```python
import json
# 测试单个样本
code = generate_code("Create a cable", "App: PowerGrid")
print(code)  # 查看实际输出
try:
    json.loads(code)
    print("✅ 有效JSON")
except:
    print("❌ 无效JSON")
```

---

## 📚 常见问题 (FAQ)

**Q: 为什么评估样本数只有50个？**
A: 为了节省内存。可以修改 `NUM_EVAL_SAMPLES = 100` 做完整评估，但需要更多显存。

**Q: 可以用CPU推理吗？**
A: 可以，但会很慢（每个样本3-5分钟）。改 `device = "cpu"` 即可。

**Q: 为什么量化后精度下降？**
A: 8-bit量化会损失<1%精度，基本无感知。模型输出质量影响不大。

**Q: 可以用更小的模型吗？**
A: 可以用CodeLlama-7B改为3B，或用其他模型如Mistral-7B。但训练时的模型配置要一致。

**Q: 评估完后模型怎么卸载？**
A: 
```python
del model, base_model, tokenizer
gc.collect()
torch.cuda.empty_cache()
```

---

In [ ]:
# 检查Google Drive和模型文件
import os
from pathlib import Path

print("🔍 诊断：检查Google Drive和模型文件")
print("=" * 70)

# 1. 检查Google Drive是否挂载
drive_path = "/content/drive/MyDrive"
if os.path.exists(drive_path):
    print(f"✅ Google Drive已挂载")
    print(f"   路径: {drive_path}")
else:
    print(f"❌ Google Drive未挂载")
    print(f"\n解决方案:")
    print(f"请运行:")
    print(f"```python")
    print(f"from google.colab import drive")
    print(f"drive.mount('/content/drive')")
    print(f"```")
    raise RuntimeError("Google Drive未挂载，请先挂载")

# 2. 检查gis-models目录
gis_models_path = "/content/drive/MyDrive/gis-models"
if os.path.exists(gis_models_path):
    print(f"\n✅ /gis-models/ 目录存在")
    print(f"   路径: {gis_models_path}")

    # 列出目录内容
    print(f"\n   📂 目录内容:")
    try:
        items = os.listdir(gis_models_path)
        for item in items:
            item_path = os.path.join(gis_models_path, item)
            if os.path.isdir(item_path):
                print(f"      📁 {item}/ (目录)")
            else:
                size_mb = os.path.getsize(item_path) / (1024*1024)
                print(f"      📄 {item} ({size_mb:.1f}MB)")
    except Exception as e:
        print(f"      ❌ 无法列出目录: {e}")
else:
    print(f"❌ /gis-models/ 目录不存在")
    print(f"   需要创建: {gis_models_path}")
    os.makedirs(gis_models_path, exist_ok=True)
    print(f"✅ 已创建目录")

# 3. 检查CodeLlama模型文件
model_path = "/content/drive/MyDrive/gis-models/codellama-gis-lora"
if os.path.exists(model_path):
    print(f"\n✅ CodeLlama模型目录存在")
    print(f"   路径: {model_path}")

    # 列出必要文件（SafeTensors 为主）
    required_files = [
        "adapter_config.json",
        "adapter_model.safetensors",  # LoRA 权重（PEFT 0.13+ 默认）
        "tokenizer.model",
        "special_tokens_map.json",
        "tokenizer_config.json"
    ]

    print(f"\n   📂 模型文件检查:")
    missing_files = []
    for f in required_files:
        file_path = os.path.join(model_path, f)
        if os.path.exists(file_path):
            size_mb = os.path.getsize(file_path) / (1024*1024)
            print(f"      ✅ {f} ({size_mb:.1f}MB)")
        else:
            # 兼容旧格式：如果缺少safetensors但存在bin，提示即可
            if f == "adapter_model.safetensors":
                legacy_bin = os.path.join(model_path, "adapter_model.bin")
                if os.path.exists(legacy_bin):
                    size_mb = os.path.getsize(legacy_bin) / (1024*1024)
                    print(f"      ✅ adapter_model.bin (legacy) ({size_mb:.1f}MB) — 建议改用 SafeTensors")
                    continue
            print(f"      ⚠️ {f} (未找到)")
            missing_files.append(f)

    if missing_files:
        print(f"\n   ⚠️ 缺失的关键文件可能影响模型加载")
else:
    print(f"\n❌ CodeLlama模型目录不存在")
    print(f"   期望位置: {model_path}")
    print(f"\n📋 可能的原因:")
    print(f"   1. 模型还没有训练完成")
    print(f"   2. 模型保存到了不同的位置")
    print(f"   3. 模型文件上传失败")
    print(f"\n💡 解决方案:")
    print(f"   选项A: 检查训练是否完成（查看步骤5-7的输出）")
    print(f"   选项B: 检查Google Drive中的实际位置")
    print(f"   选项C: 重新运行训练步骤（步骤5-7）")
    raise FileNotFoundError(f"模型文件未找到: {model_path}")

print("\n" + "=" * 70)
print("✅ 所有检查通过！可以继续加载模型")
print("=" * 70)

🔍 诊断：检查Google Drive和模型文件
✅ Google Drive已挂载
   路径: /content/drive/MyDrive

✅ /gis-models/ 目录存在
   路径: /content/drive/MyDrive/gis-models

   📂 目录内容:
      📁 codellama-gis-lora/ (目录)
      📄 codellama-gis-lora.zip (1128.1MB)

✅ CodeLlama模型目录存在
   路径: /content/drive/MyDrive/gis-models/codellama-gis-lora

   📂 模型文件检查:
      ✅ adapter_config.json (0.0MB)
      ✅ adapter_model.safetensors (305.1MB)
      ✅ tokenizer.model (0.5MB)
      ✅ special_tokens_map.json (0.0MB)
      ✅ tokenizer_config.json (0.0MB)

✅ 所有检查通过！可以继续加载模型


In [ ]:
# 从Google Drive加载已训练的模型（单GPU加载，避免offload索引错误）
print("📦 加载模型...")
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from pathlib import Path
import torch

MODEL_PATH = "/content/drive/MyDrive/gis-models/codellama-gis-lora"
BASE_MODEL = "codellama/CodeLlama-7b-Instruct-hf"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📍 模型路径: {MODEL_PATH}")
print(f"🖥️  目标设备: {device}")

# 验证路径存在
if not Path(MODEL_PATH).exists():
    raise FileNotFoundError(f"❌ 模型路径不存在: {MODEL_PATH}")

# 加载Tokenizer (使用local_files_only避免HF验证错误)
print("\n📖 加载Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    padding_side="right",
    local_files_only=True,
    trust_remote_code=True,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer加载完成")

# 加载基础模型（不使用auto offload，避免meta设备冲突）
print("\n🤖 加载基础模型...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map=None,           # 明确禁用auto offload
    low_cpu_mem_usage=True,
)
base_model.config.use_cache = False
base_model.to(device)
print("✅ 基础模型加载完成")

# 加载LoRA权重（SafeTensors / bin 自动识别）
print("\n🔧 加载LoRA权重...")
model = PeftModel.from_pretrained(
    base_model,
    MODEL_PATH,
    is_trainable=False,
    local_files_only=True,
)
model.to(device)
model.eval()
print("✅ 模型加载完成！\n")

📦 加载模型...
📍 模型路径: /content/drive/MyDrive/gis-models/codellama-gis-lora
🖥️  目标设备: cuda

📖 加载Tokenizer...
✅ Tokenizer加载完成

🤖 加载基础模型...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 快速推理测试
print("=" * 70)
print("🧪 快速推理测试")
print("=" * 70)

def generate_code(instruction, context=""):
    """使用加载的模型生成代码"""
    if context:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {context}

JSON Code:
"""
    else:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}

JSON Code:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    code = text.split("JSON Code:")[-1].strip()
    return code

# 测试案例
test_cases = [
    {
        "instruction": "Create a new MS cable object at coordinates (186355533, 439556907)",
        "context": "Application: PowerGrid | Database: ND | Steps: 5"
    },
    {
        "instruction": "Open object in editor and verify fields",
        "context": "Application: NRG Elektra | Database: elektra | Steps: 3"
    }
]

for i, test in enumerate(test_cases, 1):
    print(f"\n📝 测试案例 {i}:")
    print(f"  指令: {test['instruction']}")
    print(f"  上下文: {test['context']}")

    result = generate_code(test['instruction'], test['context'])

    # 检查JSON有效性
    try:
        json.loads(result)
        print(f"  ✅ JSON有效")
    except:
        print(f"  ⚠️ JSON格式问题")

    print(f"  💻 生成代码预览 (前200字符):\n  {result[:200]}...")

In [ ]:
# 在测试集上评估模型
print("\n" + "=" * 70)
print("📊 在测试集上评估模型")
print("=" * 70)

# 加载测试数据
print("\n📂 加载测试数据...")
with open('data/training/training_data_val.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

NUM_EVAL_SAMPLES = 50  # 评估前50个样本（快速测试）
print(f"✅ 加载了 {len(test_data)} 个样本，使用前 {NUM_EVAL_SAMPLES} 个评估")

# 定义评估指标
def is_valid_json(text):
    try:
        json.loads(text)
        return True
    except:
        return False

def get_step_count(text):
    try:
        obj = json.loads(text)
        return len(obj.get("workflow", {}).get("steps", []))
    except:
        return 0

# 开始评估
print(f"\n🧪 开始评估 ({NUM_EVAL_SAMPLES} 个样本)...\n")

json_valid_count = 0
step_counts = []
all_results = []

for i in tqdm(range(min(NUM_EVAL_SAMPLES, len(test_data))), desc="评估进度"):
    sample = test_data[i]

    try:
        instruction = sample.get("instruction", "")
        context = sample.get("input", "")

        # 生成代码
        generated_code = generate_code(instruction, context)

        # 计算指标
        is_valid = is_valid_json(generated_code)
        step_count = get_step_count(generated_code)

        if is_valid:
            json_valid_count += 1

        step_counts.append(step_count)

        all_results.append({
            "instruction": instruction,
            "json_valid": is_valid,
            "step_count": step_count
        })
    except Exception as e:
        step_counts.append(0)
        all_results.append({
            "instruction": instruction,
            "json_valid": False,
            "step_count": 0,
            "error": str(e)
        })

# 计算统计
print("\n" + "=" * 70)
print("📊 评估结果摘要")
print("=" * 70)

total_samples = len(all_results)
json_valid_rate = json_valid_count / total_samples if total_samples > 0 else 0

print(f"\n✅ JSON有效性:")
print(f"  有效: {json_valid_count}/{total_samples} ({json_valid_rate:.1%})")

print(f"\n📍 步骤数统计:")
print(f"  平均: {np.mean(step_counts):.1f}")
print(f"  中位: {np.median(step_counts):.1f}")
print(f"  最大: {np.max(step_counts):.0f}")
print(f"  最小: {np.min(step_counts):.0f}")

print(f"\n🎯 总体质量评估: {'⭐⭐⭐⭐' if json_valid_rate > 0.7 else '⭐⭐⭐' if json_valid_rate > 0.5 else '⭐⭐'}")

print("\n" + "=" * 70)